In [11]:
import sys
sys.path.append('../../../')
import helper

import numpy as np
import matplotlib.pyplot as plt

In [12]:
results = helper.get_filenames_results()
for key in results:
    print(helper.get_params_results(key))

{'dataset_name': '3', 'method': 'gauss-newton', 'init_dist_x': 50, 'init_dist_y': 0, 'init_density_x': 101, 'init_density_y': 1}


In [13]:
def print_results_one_line(results_list, labels_list, axis=0):
    for i in range(len(results_list)):
        plt.plot(results_list[i][:, axis], results_list[i][:, 2], label=labels_list[i])
    plt.xlabel(f'init point axis={axis}')
    plt.ylabel('Steps')
    plt.title('Research starting point')
    plt.grid(True)
    plt.legend()
    plt.show()

In [14]:
def get_and_print_result(name_lists, axis):
    results_list = []
    labels_list = []
    for name in name_lists:
        research = helper.load_matrix(results[name])
        results_list = [research]
        labels_list = [helper.get_params_results(name)['method']]

    print_results_one_line(results_list, labels_list, axis=axis)

In [15]:
get_and_print_result(['1'], 0)

ValueError: invalid literal for int() with base 10: '['